In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("working_csv.csv")
Y = df["P. Habitable Class"]
X = df.drop("P. Habitable Class",axis=1)

header = list(df.columns)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

X_train_list = X_train.values.tolist()
X_test_list = X_test.values.tolist()
Y_train_list = Y_train.values.tolist()
Y_test_list = Y_test.values.tolist()
    
training_data = list(map(lambda x:x[0]+[x[1]],zip(X_train_list,Y_train_list)))
testing_data = list(map(lambda x:x[0]+[x[1]],zip(X_test_list,Y_test_list)))

X.head()

,P. Name,P. Zone Class,P. Mass Class,P. Composition Class,P. Atmosphere Class,P. Density (EU),P. Gravity (EU),P. Esc Vel (EU),P. SFlux Min (EU),P. SFlux Mean (EU),...,P. HZD,P. HZC,P. HZA,P. HZI,P. SPH,P. ESI,S. HabCat,P. Habitable,P. Hab Moon,P. Confirmed
0,GJ 163 c,Warm,Superterran,rocky-iron,metals-rich,1.19,2.17,1.99,0.551557,0.663002,...,-0.34,-0.14,0.81,0.53,0.08,0.73,0,1,0,1
1,GJ 667 C e,Warm,Terran,rocky-iron,metals-rich,0.99,1.39,1.39,0.290249,0.302030,...,0.51,-0.16,0.23,0.63,0.00,0.60,0,1,0,0
2,GJ 667 C f,Warm,Terran,rocky-iron,metals-rich,0.99,1.39,1.39,0.530653,0.563207,...,-0.22,-0.16,0.08,0.78,0.00,0.77,0,1,0,0
3,GJ 682 c,Warm,Superterran,rocky-iron,hydrogen-rich,1.24,2.37,2.13,0.302060,0.367301,...,0.23,-0.14,1.19,0.45,0.00,0.59,0,1,0,0
4,GJ 3293 d,Warm,Superterran,rocky-iron,metals-rich,1.20,2.22,2.03,0.447687,0.565605,...,-0.21,-0.14,0.91,0.52,0.00,0.70,0,1,0,1


In [3]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [4]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [5]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [6]:
count = class_counts(training_data)
count

{'mesoplanet': 13, 'non-habitable': 11, 'psychroplanet': 13}

In [7]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [8]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def __repr__(self):
        condition = ">="
        if not is_numeric(self.value):
            condition = "=="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    

In [9]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [10]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):

        values = set([row[col] for row in rows])

        for val in values:

            question = Question(col, val)

            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [11]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [12]:
class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [13]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [14]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "└── Predict", node.predictions)
        return
    print (spacing + str(node.question))

    print (spacing + '├── True:')
    print_tree(node.true_branch, spacing + "|  ")

    print (spacing + '├── False:')
    print_tree(node.false_branch, spacing + "|  ")

In [15]:
my_tree = build_tree(training_data)

In [16]:
print_tree(my_tree)

Is P. SPH >= 0.53?
├── True:
|  Is S. Hab Zone Max (AU) >= -0.39?
|  ├── True:
|  |  └── Predict {'psychroplanet': 13}
|  ├── False:
|  |  └── Predict {'mesoplanet': 13}
├── False:
|  └── Predict {'non-habitable': 11}


In [17]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [18]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [19]:
y_pred = []
y_test = []
for i,row in enumerate(testing_data):
#     print ("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))
    val = classify(row,my_tree)
    
    y_test.append(row[-1])
    y_pred.append(list(val.keys())[0])

In [20]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[5 0 0]
 [0 7 0]
 [1 0 4]]
               precision    recall  f1-score   support

   mesoplanet       0.83      1.00      0.91         5
non-habitable       1.00      1.00      1.00         7
psychroplanet       1.00      0.80      0.89         5

     accuracy                           0.94        17
    macro avg       0.94      0.93      0.93        17
 weighted avg       0.95      0.94      0.94        17

